# How to use a SmartLLMChain

A SmartLLMChain is a form of self-critique chain that can help you if have particularly complex questions to answer. Instead of doing a single LLM pass, it instead performs these 3 steps:
1. Ideation: Pass the user prompt n times through the LLM to get n output proposals (called "ideas"), where n is a parameter you can set 
2. Critique: The LLM critiques all ideas to find possible flaws and picks the best one 
3. Resolve: The LLM tries to improve upon the best idea (as chosen in the critique step) and outputs it. This is then the final output.

SmartLLMChains are based on the SmartGPT workflow proposed in https://youtu.be/wVzuvf9D9BU.

Note that SmartLLMChains
- use more LLM passes (ie n+2 instead of just 1)
- only work then the underlying LLM has the capability for reflection, which smaller models often don't
- only work with underlying models that return exactly 1 output, not multiple

This notebook demonstrates how to use a SmartLLMChain.

##### Same LLM for all steps

In [4]:
from langchain.prompts import PromptTemplate
from langchain_experimental.smart_llm import SmartLLMChain
from langchain_community.chat_models.gigachat import GigaChat

В качестве примера взята задача из оригинальной публикации SmartGPT. Это пример из оригинального видео: (https://youtu.be/wVzuvf9D9BU?t=384). Замечено, что не смотря на то, что задача очень простая, LLM крайне плохо справляются с решением подобных задач.

Как мы видим, большая часть изначальных решений неверная, но за счет решений и цепочки рассуждений, модели в итоге удаётся прийти к правильному ответу.

In [42]:
hard_question = """У меня есть пустые канистры на 6 и 12 литров и кран с водой. Мне нужно, чтобы в большой канистре оказалось 6 литров жидкости. Как это сделать?"""

So, we first create an LLM

In [27]:
llm = GigaChat(credentials="...")

Now we can create a SmartLLMChain

In [43]:
prompt = PromptTemplate.from_template(hard_question)
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=5, verbose=True)

Now we can use the SmartLLM as a drop-in replacement for our LLM. E.g.:

In [44]:
chain.invoke({})



> Entering new SmartLLMChain chain...
Prompt after formatting:
У меня есть пустые канистры на 6 и 12 литров и кран с водой. Мне нужно, чтобы в большой канистре оказалось 6 литров жидкости. Как это сделать?
Idea 1:
Хорошо, давайте разберемся. У нас есть две канистры - одна на 6 литров, а другая на 12 литров. Наша цель - заполнить большую канистру на 12 литров 6 литрами жидкости. 

Шаг 1: Сначала мы должны заполнить большую канистру на 12 литров водой. Для этого мы открываем кран и начинаем наливать воду в канистру, пока она не заполнится до 12 литров.

Шаг 2: Затем мы берем пустую канистру на 6 литров и начинаем переливать воду из большой канистры в маленькую. Мы продолжаем это делать, пока маленькая канистра не заполнится до 6 литров.

Шаг 3: После того, как маленькая канистра заполнится до 6 литров, мы можем закрыть кран и оставить большую канистру пустой.

Таким образом, мы достигли нашей цели - заполнили большую канистру на 12 литров 6 литрами жидкости.
Idea 2:
Для начала, давайте

{'resolution': 'Ответ:\n\nИсследователь считает, что лучшим вариантом ответа является вариант 5. Он предлагает заполнить маленькую канистру на 6 литров водой, а затем перелить эту воду в большую канистру на 12 литров. Таким образом, в большой канистре окажется 6 литров жидкости.\n\nУлучшенный ответ:\n\nДля того чтобы заполнить большую канистру на 12 литров до 6 литров, нужно сначала заполнить маленькую канистру на 6 литров водой. Затем, используя воронку или другой подходящий инструмент, перелить воду из маленькой канистры в большую канистру до достижения нужного объема.\n\nПолный ответ:\n\nДля того чтобы заполнить большую канистру на 12 литров до 6 литров, нужно сначала заполнить маленькую канистру на 6 литров водой. Затем, используя воронку или другой подходящий инструмент, перелить воду из маленькой канистры в большую канистру до достижения нужного объема.'}

##### Different LLM for different steps

You can also use different LLMs for the different steps by passing `ideation_llm`, `critique_llm` and `resolve_llm`. You might want to do this to use a more creative (i.e., high-temperature) model for ideation and a more strict (i.e., low-temperature) model for critique and resolution.

In [8]:
chain = SmartLLMChain(
    ideation_llm=ChatOpenAI(temperature=0.9, model_name="gpt-4"),
    llm=ChatOpenAI(
        temperature=0, model_name="gpt-4"
    ),  # will be used for critique and resolution as no specific llms are given
    prompt=prompt,
    n_ideas=3,
    verbose=True,
)